### Churn Modelling

- In this model, we have to predict if a bank customer will take an exit in the future or not.  

In [2]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# import dataset

dataset = pd.read_csv("C:/Users/Ashish/Desktop/Deep Learning/CSV files/Churn_Modelling.txt", sep = ",")

In [6]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
dataset.shape

(10000, 14)

- There are  10,000 rows and 13 columns in this dataset
- The first three columns do not provide any data and can be removed
- The last column/feature [Exited] is our dependent feature
- Remaining 9 columns/features are our independent features

In [8]:
# Creating dependent and independent feature dataset

X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [9]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [10]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

- Geography and Gender are categorical features. Hence we have to convert them to numerical features 

In [11]:
# Create dummy variables
geography = pd.get_dummies(X["Geography"], drop_first = True)
gender = pd.get_dummies(X["Gender"], drop_first = True)

In [12]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [13]:
geography.head()

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1


In [14]:
gender.head()

,Male
0,0
1,0
2,0
3,0
4,0


In [16]:
# Concatenate dummy variables to the dataframe

X = pd.concat([X, geography, gender], axis = 1)

In [17]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,1,0


- Now, We do not need Geography and Gender categorical columns. Hence we drop them

In [19]:
## Drop Unwanted categorical columns

X = X.drop(["Geography", "Gender"], axis = 1)

In [20]:
# Splitting the data into training and testing data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#### Need of Feature Scaling for Deep learning

1. Yes, we require feature scaling in Deep learning. 
2. The reason we need to scale the input is, in the hidden layer, the features get multiplied by the weights.
3. If the features are of different magnitudes, the time taken for multiplication will be very high.
4. Also, during back propagation, the time taken to find the derivative also increases. Hence the weights take more time to converge to global minima
5. Hence we should scale all the features to the same scale range.

In [22]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [24]:
X_train

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.5698444 ,
        -0.57369368,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ...,  1.75486502,
        -0.57369368, -1.09168714]])

### Lets Start Making ANN!

- Every neural network that you create, whether it is ANN, CNN or RNN, we have to import Sequential
- Sequential library is responsible for creating these neural networks
- To create hidden layer we have to import Dense library
- Import activation functions
- Import Dropout layer

In [25]:
# Importing Keras Libraries and Packages

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LeakyReLU, PReLU, ELU

In [26]:
# Initializing The ANN

classifier = Sequential() # This is an empty neural network

In [28]:
# Adding the input layer and first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = "he_uniform", activation = "relu", input_dim = 11))

- units = number of neurons in the first hidden layer
- input_dim = number of input features
- kernel_initialization = weight initialization

In [29]:
# Adding the second hidden layer

classifier.add(Dense(units = 6, kernel_initializer = "he_uniform", activation = "relu"))

In [30]:
# Adding the output layer

classifier.add(Dense(units = 1, kernel_initializer = "glorot_uniform", activation = "sigmoid"))

- Since we have a binary output ie 0 or 1, thereform no. of neurons used in output layer is 1

In [32]:
# Compiling the ANN
classifier.compile(optimizer="Adamax", loss = "binary_crossentropy", metrics = ["accuracy"])

In [34]:
# Fitting the ANN to Training Set

model_history = classifier.fit(X_train, y_train, validation_split= 0.33, batch_size= 10, nb_epoch = 100)

Train on 5359 samples, validate on 2641 samples
Epoch 1/100
 400/5359 [=>............................] - ETA: 1s - loss: 0.4101 - accuracy: 0.8200

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


5359/5359 [==============================] - 3s 533us/step - loss: 0.3914 - accuracy: 0.8337 - val_loss: 0.4099 - val_accuracy: 0.8220
Epoch 2/100
5359/5359 [==============================] - 2s 435us/step - loss: 0.3893 - accuracy: 0.8339 - val_loss: 0.4080 - val_accuracy: 0.8232
Epoch 3/100
5359/5359 [==============================] - 3s 505us/step - loss: 0.3877 - accuracy: 0.8337 - val_loss: 0.4068 - val_accuracy: 0.8228
Epoch 4/100
5359/5359 [==============================] - 3s 477us/step - loss: 0.3860 - accuracy: 0.8356 - val_loss: 0.4059 - val_accuracy: 0.8228
Epoch 5/100
5359/5359 [==============================] - 3s 493us/step - loss: 0.3848 - accuracy: 0.8388 - val_loss: 0.4046 - val_accuracy: 0.8236
Epoch 6/100
5359/5359 [==============================] - 3s 500us/step - loss: 0.3832 - accuracy: 0.8384 - val_loss: 0.4040 - val_accuracy: 0.8239
Epoch 7/100
5359/5359 [==============================] - 3s 537us/step - loss: 0.3821 - accuracy: 0.8405 - val_loss: 0.4023 - val_

5359/5359 [==============================] - 2s 460us/step - loss: 0.3396 - accuracy: 0.8587 - val_loss: 0.3645 - val_accuracy: 0.8504
Epoch 57/100
5359/5359 [==============================] - 3s 468us/step - loss: 0.3393 - accuracy: 0.8614 - val_loss: 0.3641 - val_accuracy: 0.8508
Epoch 58/100
5359/5359 [==============================] - 3s 505us/step - loss: 0.3386 - accuracy: 0.8602 - val_loss: 0.3643 - val_accuracy: 0.8501
Epoch 59/100
5359/5359 [==============================] - 2s 435us/step - loss: 0.3381 - accuracy: 0.8606 - val_loss: 0.3633 - val_accuracy: 0.8493
Epoch 60/100
5359/5359 [==============================] - 3s 496us/step - loss: 0.3379 - accuracy: 0.8614 - val_loss: 0.3630 - val_accuracy: 0.8493
Epoch 61/100
5359/5359 [==============================] - 3s 472us/step - loss: 0.3375 - accuracy: 0.8615 - val_loss: 0.3628 - val_accuracy: 0.8504
Epoch 62/100
5359/5359 [==============================] - 3s 483us/step - loss: 0.3374 - accuracy: 0.8614 - val_loss: 0.3621 

In [36]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 7         
Total params: 163
Trainable params: 163
Non-trainable params: 0
_________________________________________________________________


- Total Params means number or weights and bias

In [38]:
# Predicting the test set results

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [39]:
# making Confusion Matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred) 

In [40]:
cm

array([[1536,   59],
       [ 219,  186]], dtype=int64)

In [41]:
# Calculate Test accuracy

from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)

In [42]:
score

0.861